In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
from torch.autograd import Variable
import torch.nn.functional as F

from torchvision import datasets, transforms
from utils import build_dataset
from tqdm import tqdm_notebook

from tensorboardX import SummaryWriter
writer = SummaryWriter('runs/MobileNet')

from flops_counter import *
from nets.MobileNet_CIFAR import MobileNet_CIFAR

In [2]:
class Solver(object):
    def __init__(self, model, dataset='MNIST', n_epochs=100, lr=0.001):
        self.n_epochs = n_epochs
        
        self.train_loader, self.test_loader = build_dataset(dataset, './data', batch_size = 300)
        
        self.image_dim = {'MNIST': 28*28, 'CIFAR10': 3*32*32}[dataset]
                
        self.net = model().cuda()
        self.net = add_flops_counting_methods(self.net)
        self.loss_fn = nn.CrossEntropyLoss()
        self.optimizer = optim.Adam(self.net.parameters(), lr=lr)
                
    def train(self):
        self.net.train()
        best_loss = 999
        for epoch_i in tqdm_notebook(range(self.n_epochs)):
            epoch_loss = 0
            for batch_idx, (images, labels) in enumerate(self.train_loader):
                images = Variable(images).cuda()
                labels = Variable(labels).cuda()
                
                logits = self.net(images)
                
                loss = self.loss_fn(logits, labels)
                
                total_loss = loss

                self.optimizer.zero_grad()
                total_loss.backward()

                self.optimizer.step()
                
                epoch_loss += float(loss.data)    
            epoch_loss /= len(self.train_loader.dataset)
            
            if best_loss > epoch_loss:
                best_loss = epoch_loss
                torch.save(self.net.state_dict(), 'models/MobileNet_CIFAR.pth')
                
            print(f'Epoch {epoch_i} | loss: {epoch_loss:.8f}')
            
            niter = epoch_i*len(self.train_loader)+batch_idx
            writer.add_scalar('Train/Loss', loss.data[0], niter)
            self.evaluate(niter)
            
        
        self.net.load_state_dict(torch.load(PATH))
        print (f'Best Loss {best_loss} | Best')
        self.evaluate().start_flops_count()
    
    def evaluate(self, niter=None):
        total = 0
        correct = 0
        self.net.eval()
        for batch_idx, (images, labels) in enumerate(self.test_loader):
            images = Variable(images).cuda()

            logits = self.net(images.cuda())
            
            _, predicted = torch.max(logits.data, 1)
            
            total += labels.size(0)
            correct += (predicted.cpu() == labels).sum()
            
            if niter == None:
                writer.add_scalar('Test/Acc@1', correct / total, batch_idx)
            else:
                writer.add_scalar('Train/Acc@1', correct / total, niter)
        print('Flops:  {}'.format(flops_to_string(self.net.compute_average_flops_cost())))
        print('Params: ' + get_model_parameters_number(self.net))
        
        print(f'Accuracy: {100 * correct / total:.3f}%')

In [ ]:
standard_solver = Solver(MobileNet_CIFAR, 'CIFAR10')

standard_solver.train()

Files already downloaded and verified
Files already downloaded and verified


Epoch 0 | loss: 0.00603249


/home/denis/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:44: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


Flops:  0Mac
Params: 121.61k
Accuracy: 43.000%
Epoch 1 | loss: 0.00540838
Flops:  0Mac
Params: 121.61k
Accuracy: 46.000%
Epoch 2 | loss: 0.00454111
Flops:  0Mac
Params: 121.61k
Accuracy: 51.000%
Epoch 3 | loss: 0.00413872
Flops:  0Mac
Params: 121.61k
Accuracy: 54.000%
Epoch 4 | loss: 0.00380992
Flops:  0Mac
Params: 121.61k
Accuracy: 59.000%
Epoch 5 | loss: 0.00356381
Flops:  0Mac
Params: 121.61k
Accuracy: 60.000%
Epoch 6 | loss: 0.00340752
Flops:  0Mac
Params: 121.61k
Accuracy: 63.000%
Epoch 7 | loss: 0.00322644
Flops:  0Mac
Params: 121.61k
Accuracy: 62.000%
Epoch 8 | loss: 0.00309881
Flops:  0Mac
Params: 121.61k
Accuracy: 65.000%
